#### Fase 4

In [1]:
#LIBRERIAS PARA MANIPULACION DE TABLAS DE DATOS
import pandas as pd
import numpy as np
import sqlite3
import openpyxl

In [2]:
link = r"C:\Users\rodmo\Desktop\MBD Maestría EGADE\Gestión de Bases de Datos y Computación en la Nube\Archivo Local Laptop Gestión de Bases de Datos y Computación en la Nube\Twitter Scraping Results.xlsx"
publicacion = pd.read_excel(link, sheet_name="publicacion")
comentario = pd.read_excel(link, sheet_name="comentario")
red_social = pd.read_excel(link, sheet_name="red_social")

In [3]:
print(len(publicacion))
publicacion = publicacion.dropna(subset=['Fecha_publicacion'])
print(len(publicacion))

212
211


In [4]:
print(len(comentario))
comentario = comentario.dropna(subset=['Fecha_comentario'])
print(len(comentario))

755
753


In [5]:
#publicacion['contenido_publicacion'] = publicacion['contenido_publicacion'].str.replace(' ', '')
#comentario['contenido_comentario'] = comentario['contenido_comentario'].str.replace(' ', '')

In [6]:
publicacion.head()

,ID_publicacion,ID_red_social,URL,contenido_publicacion,Fecha_publicacion
0,P-01,1,https://x.com/FranFijap/status/187818521158376...,fran fijap@franfijap sheinbaum en el camino de...,"Jan 11, 2025"
1,P-02,1,https://x.com/AlmMiguelito/status/188207714325...,miguel almeida@almmiguelito@lorenzocordovav me...,"Jan 22, 2025"
2,P-03,1,https://x.com/revistaetcetera/status/192744946...,etcétera@revistaetceteraconfesado por ella mis...,"May 27, 2025"
3,P-04,1,https://x.com/SinLinea_Mx/status/1894447267626...,sinlínea.mx@sinlinea_mxcontundente #sheinbaum...,"Feb 25, 2025"
4,P-05,1,https://x.com/AtypicalTeve/status/188069968129...,atypical te ve@atypicalteve sheinbaum en probl...,"Jan 18, 2025"


In [7]:
comentario.head()

,ID_comentario,ID_publicacion,contenido_comentario,Fecha_comentario,URL
0,Ch-01,P-01,thelibertarian@marcolouvierpor eso en estados ...,"Jan 11, 2025",https://x.com/MarcoLouvier/status/187820216848...
1,Ch-02,P-01,praetor del león@praetorleonya los vas a tener...,"Jan 11, 2025",https://x.com/PraetorLeon/status/1878186741036...
2,Ch-03,P-01,beto serna@senneville12es normal que los presi...,"Jan 11, 2025",https://x.com/Senneville12/status/187818663764...
3,Ch-04,P-01,biondachoripanera@choribiondaes lo que votaron...,"Jan 11, 2025",https://x.com/ChoriBionda/status/1878186624313...
4,Ch-05,P-02,paz con dialogo@pazcondialogoparody accountnad...,"Jan 22, 2025",https://x.com/PazConDialogo/status/18820775142...


In [8]:
red_social.head()

,ID_red_social,Nombre
0,1,Twitter
1,2,Facebook
2,3,Instagram
3,4,Otros medios


In [9]:
##################################################
##################################################
####################BRONZE LAYER##################
##################################################
##################################################

In [10]:
#CONECTAR AL SISTEMA SQLITE Y CREAR LA BASE DE DATOS VENTAS
conn = sqlite3.connect("Sentiment_Analysis.db")
cursor = conn.cursor()

In [11]:
#CREAR TABLA CLIENTES EN LA BASE DE DATOS
cursor.execute("DROP TABLE IF EXISTS bronze_publicacion;")
cursor.execute("""
CREATE TABLE bronze_publicacion (
    ID_publicacion TEXT PRIMARY KEY,
    ID_red_social INTEGER,
    URL TEXT,
    contenido_publicacion TEXT,
    Fecha_publicacion TEXT
);
""")

publicacion.to_sql('bronze_publicacion', conn, if_exists='append', index=False)

211

In [12]:
#CREAR TABLA CLIENTES EN LA BASE DE DATOS
cursor.execute("DROP TABLE IF EXISTS bronze_comentario;")
cursor.execute("""
CREATE TABLE bronze_comentario (
    ID_comentario TEXT PRIMARY KEY,
    ID_publicacion TEXT,
    contenido_comentario TEXT,
    Fecha_comentario TEXT,
    URL TEXT
);
""")

comentario.to_sql('bronze_comentario', conn, if_exists='append', index=False)

753

In [13]:
#CREAR TABLA CLIENTES EN LA BASE DE DATOS
cursor.execute("DROP TABLE IF EXISTS bronze_red_social;")
cursor.execute("""
CREATE TABLE bronze_red_social (
    ID_red_social TEXT PRIMARY KEY,
    Nombre TEXT
);
""")

red_social.to_sql('bronze_red_social', conn, if_exists='append', index=False)

4

In [14]:
##################################################
##################################################
####################SILVER LAYER##################
##################################################
##################################################

In [15]:
#CREACION DE TABLA DE TRANSACCIONES CON INFROMACION DE CLIENTE Y PRODUCTOS
cursor.execute("DROP TABLE IF EXISTS silver_publicacion;")
cursor.execute("""
CREATE TABLE silver_publicacion AS
SELECT
    p.ID_publicacion,
    c.ID_comentario,
    r.Nombre AS red_social_name,
    p.Fecha_publicacion AS Fecha,
    p.contenido_publicacion AS publicacion,
    c.contenido_comentario AS comentario

FROM bronze_publicacion p
LEFT JOIN bronze_comentario c on c.ID_publicacion = p.ID_publicacion
LEFT JOIN bronze_red_social r ON r.ID_red_social = p.ID_red_social;
""")

In [16]:
#VISUALIZAR UNA TABLA DADA
visualize = pd.read_sql_query("SELECT * FROM silver_publicacion", conn)
visualize.head(2)

,ID_publicacion,ID_comentario,red_social_name,Fecha,publicacion,comentario
0,P-01,Ch-01,Twitter,"Jan 11, 2025",fran fijap@franfijap sheinbaum en el camino de...,thelibertarian@marcolouvierpor eso en estados ...
1,P-01,Ch-02,Twitter,"Jan 11, 2025",fran fijap@franfijap sheinbaum en el camino de...,praetor del león@praetorleonya los vas a tener...


In [17]:
##################################################
##################################################
####################GOLD LAYER##################
##################################################
##################################################

In [18]:
#CREACION DE TABLA ORO CON EL PROMEDIO DE TRANSACCIONES
cursor.execute("DROP TABLE IF EXISTS gold_publicacion;")
cursor.execute("""
CREATE TABLE gold_publicacion AS
SELECT
    ID_publicacion,
    GROUP_CONCAT(ID_comentario) AS ID_comentarios_list,
    red_social_name,
    Fecha,
    publicacion,
    CONCAT(comentario) AS comentarios,
    COUNT(ID_comentario) AS total_comentarios


FROM silver_publicacion
GROUP BY ID_publicacion;
""")

In [19]:
visualize2 = pd.read_sql_query("SELECT * FROM gold_publicacion", conn)
visualize2.head(5)

,ID_publicacion,ID_comentarios_list,red_social_name,Fecha,publicacion,comentarios,total_comentarios
0,P-01,"Ch-01,Ch-02,Ch-03,Ch-04",Twitter,"Jan 11, 2025",fran fijap@franfijap sheinbaum en el camino de...,thelibertarian@marcolouvierpor eso en estados ...,4
1,P-010,"Ch-043,Ch-044,Ch-045,Ch-046",Twitter,"Apr 29, 2025",viviana hinojosa@vivianolalaentre el cartón de...,schadenfreude milk@schadenfreudefuno puedes am...,4
2,P-0100,"Ch-0374,Ch-0375,Ch-0376,Ch-0377",Twitter,"Mar 17, 2025",jorge fernández menéndez@j_fdz_menendezla falt...,d'angelo 4t@angmariscal4tcon una pedante pasmo...,4
3,P-0101,"Ch-0378,Ch-0379,Ch-0380,Ch-0381,Ch-0382",Twitter,"Aug 29, 2024",periódico supremo@diario_supremo #últimahora |...,ᗰᖇ. ᑕíᐯiᑕo ®₿@mr_civicoel culiacanazo,5
4,P-0102,"Ch-0383,Ch-0384,Ch-0385,Ch-0386",Twitter,"Aug 26, 2024",general toribio ortega@tori_ort1. sheinbaum s...,general toribio ortega@tori_ort2. preocupada...,4


In [21]:
excel_file = link.replace("Twitter Scraping Results.xlsx", "Twitter_Scraping_Gold_DB_v2.xlsx")
visualize2.to_excel(excel_file)